In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

In [2]:

from keras.applications.vgg16 import preprocess_input

In [3]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [128, 128]

In [7]:
from tensorflow.keras.applications import ResNet50

In [9]:
import tensorflow as tf

In [10]:
Resnet=tf.keras.applications.ResNet50(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 27s 0us/step


In [12]:
for layer in Resnet.layers:
  layer.trainable = False

In [14]:
x = Flatten()(Resnet.output)
prediction = Dense(4, activation='softmax')(x)

In [15]:
prediction.shape

(None, 4)

In [16]:
# create a model object
model = Model(inputs=Resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 23,718,788 (90.48 MB)

 Trainable params: 131,076 (512.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [17]:
import os
import cv2

In [18]:
def load_images(directory_path, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory_path, filename)
            img = load_image(img_path, target_size=target_size)  
            images.append(img)
    return np.array(images)



def load_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    img = cv2.resize(img, target_size)
    return img



In [19]:
covid_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//COVID//images'
opacity_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Lung_Opacity//images'
normal_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Normal//images'
pneumonia_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Viral Pneumonia//images'


covid_images = load_images(covid_images_dir)
opacity_images = load_images(opacity_images_dir)
normal_images = load_images(normal_images_dir)
pneumonia_images = load_images(pneumonia_images_dir)


X = np.concatenate([covid_images, opacity_images, normal_images, pneumonia_images], axis=0)

In [20]:
covid_label = 0
opacity_label = 1
normal_label = 2
pneumonia_label = 3

covid_labels = np.full((len(covid_images),), covid_label)
opacity_labels = np.full((len(opacity_images),), opacity_label)
normal_labels = np.full((len(normal_images),), normal_label)
pneumonia_labels = np.full((len(pneumonia_images),), pneumonia_label)

In [21]:
Y = np.concatenate([covid_labels, opacity_labels, normal_labels, pneumonia_labels], axis=0)


In [22]:
shuffle_index = np.random.permutation(len(X))
X_ = X[shuffle_index]
Y_ = Y[shuffle_index]

In [23]:
from keras.utils import to_categorical

# Assuming Y_train is loaded or generated with integer labels
Y_ = to_categorical(Y_, num_classes=4)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.2, random_state=42)

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(f"Y_train shape before: {Y_train.shape}") 
history = model.fit(X_train, Y_train, batch_size=16, epochs=10, validation_data=(X_test, Y_test))
train_loss, train_accuracy = model.evaluate(X_train, Y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Y_train shape before: (16932, 4)
Epoch 1/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.7926 - loss: 2.4456

KeyboardInterrupt: 